# Analyzing Relative Performance of Different DP Methods

## Activate Project and Import Functions

In [1]:
] activate ..

  Activating project at `~/Documents/Class/ThAlgs/Edist`


In [2]:
using Edist, BenchmarkTools

## Read FASTA Input Data

In [3]:
seq = get_fasta("../data/TP53_cross_species.fasta")

300-element Vector{String}:
 "CTCAAAAGTCTAGAGCCACCGTCCAGGGAG" ⋯ 2452 bytes ⋯ "CTCTTATTTTACAATAAAACTTTGCTGCCA"
 "TTTCCCCTCCCACGTGCTCACCCTGGCTAA" ⋯ 1721 bytes ⋯ "ATAAAATTTTGTTATCACTTAAAAAAAAAA"
 "TCTGAAGCTCCAGTTCATTGGGACTTATCC" ⋯ 1732 bytes ⋯ "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
 "CTGTAACTAGGGGAATCCCCAAAACTCCAC" ⋯ 2176 bytes ⋯ "TCTTTAAAAAAAAAAAAAAAAAAAAAAAAA"
 "TAAAGTCCAGAGCCACCATCCTGGCCGCAG" ⋯ 2115 bytes ⋯ "AAAACTTTGCTACCAAAAAAAAAAAAAAAA"
 "GAATTCCGAACGGCGGCGGCGGCGGCGGCG" ⋯ 1495 bytes ⋯ "TTCTTTTTTTACTGAAAAAAAAAGGAATTC"
 "AAAAGTCCAGGGCCACCATCCTTGGCGCAG" ⋯ 1789 bytes ⋯ "TAGTGTGGTAGCAAAAAAAAAAAAAAAAAA"
 "AGTTTTGAGCTTCTCAAAAGTCCAGAGCCA" ⋯ 2050 bytes ⋯ "TCTCTTATTTTACAATAAAACTTTACTACC"
 "GCCGAGAGGAACGGACCGGATAGAAACAGA" ⋯ 2348 bytes ⋯ "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
 "CCATCCTGGCCGCAGTGGCCGCTGGTCTCC" ⋯ 2095 bytes ⋯ "AACTTTGCTACCAAAAAAAAAAAAAAAAAA"
 "CTCAAAAGTCTAGAGCCACCGTCCAGGGAG" ⋯ 2574 bytes ⋯ "CTCTTATTTTACAATAAAACTTTGCTGCCA"
 "GCGGCCGCGGCGCAGGTAGCTGCTGGGCTC" ⋯ 2083 bytes ⋯ "TCCCCTTTTTATATCGGTTT

In [4]:
@benchmark score(Bounded, seq[1], seq[2])

BenchmarkTools.Trial: 540 samples with 1 evaluation.
 Range (min … max):  8.822 ms …  12.155 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     9.142 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.258 ms ± 360.677 μs  ┊ GC (mean ± σ):  1.16% ± 2.72%

      ▃▅▇▇█▇▇▄▅▁▂       ▃▂▃▂                                   
  ▆▆▄█████████████▄▆▆▆██████▆▆▆▄▄▄▆▄▆▁▄▄▁▁▁▁▄▁▄▁▁▁▁▁▁▄▁▄▁▁▆▁▆ █
  8.82 ms      Histogram: log(frequency) by time      10.9 ms <

 Memory estimate: 10.01 MiB, allocs estimate: 9.

In [5]:
@benchmark score(Full, seq[1], seq[2])

BenchmarkTools.Trial: 231 samples with 1 evaluation.
 Range (min … max):  20.198 ms … 24.495 ms  ┊ GC (min … max): 0.00% … 15.05%
 Time  (median):     20.588 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   21.655 ms ±  1.652 ms  ┊ GC (mean ± σ):  5.42% ±  6.95%

   ▁▃▅█▄                                                       
  ▅██████▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▄▄▅█▇▅▄▄▄▃▃ ▃
  20.2 ms         Histogram: frequency by time        24.4 ms <

 Memory estimate: 34.17 MiB, allocs estimate: 2.